In [266]:
import sys

import time
import pandas as pd
import numpy as np


#   paths
ANKI_PATH = 'C:/Users/plk2033/Documents/PythonDocs/anki-master'
COLLECTION_PATH = "C:/Users/plk2033/Documents/PythonDocs/collection.anki2"
pd.options.display.float_format = '{:.0f}'.format
sys.path.append(ANKI_PATH)

import anki
import math
from anki import Collection
from anki.utils import intTime

In [159]:
col = Collection(COLLECTION_PATH)

cards = col.db.all("select id,nid,did from cards")

col.close()

In [160]:
col = Collection(COLLECTION_PATH)

card_id = []
note_id = []
tags =[]

for card in cards:
    card_id.append(card[0])
    note_id.append(card[1])
    tags.append(col.db.all("select tags from notes where id = " + str(card[1]))[0][0].split())
    
cards_w_tags = pd.DataFrame(list(zip(card_id, note_id,tags)), 
            columns =['Card_ID', 'Note_ID','Tags'])

col.close()

In [161]:
col = Collection(COLLECTION_PATH)

revlog = col.db.all("select id,cid,ease from revlog")

col.close()

In [286]:
date = []
card_id = []
ease =[]

result=[]
for card in revlog:
    date.append(time.strftime('%Y-%m-%d', time.localtime(card[0]/1000)))
    card_id.append(card[1])
    ease.append(card[2])

revlog_cards = pd.DataFrame(list(zip(date, card_id,ease)), 
            columns =['Date', 'Card_ID','Ease'])
revlog_cards = revlog_cards[revlog_cards['Date'] >= '2018-01-01'] 

In [287]:
revlog_cards = pd.merge(revlog_cards, cards_w_tags, how='left', on=['Card_ID'])

In [288]:
#revlog_cards[pd.isnull(revlog_cards).any(axis=1)]
revlog_cards = revlog_cards[np.isfinite(revlog_cards['Note_ID'])]

In [290]:
def f(row):
    if row['Ease'] < 3:
        val = 0
    else:
        val = 1
    return val

revlog_cards['Correct'] = revlog_cards.apply(f, axis=1)

In [291]:
max_len = 0
index = 0
for item in revlog_cards['Tags']:
    if len(item) > max_len:
        max_len = len(item)
    index = index+1

print(max_len)

4


In [292]:
revlog_cards[['Tag1','Tag2','Tag3','Tag4']] = pd.DataFrame(
    revlog_cards.Tags.values.tolist(), index= revlog_cards.index)

In [296]:
 def exam(row):
    if any("Exam7" in s for s in row['Tags']):
        val = 'Exam7'
    elif any("Exam8" in s for s in row['Tags']):
        val = 'Exam8'
    elif any("Exam9" in s for s in row['Tags']):
        val = 'Exam9'
    else:
        val = 'Other'
    return val

revlog_cards['Exam'] = revlog_cards.apply(exam, axis=1)

In [306]:
Tag_list = revlog_cards['Tag1'].unique().tolist() + revlog_cards['Tag2'].unique().tolist() + revlog_cards[
    'Tag3'].unique().tolist() + revlog_cards['Tag4'].unique().tolist()
Tag_list = list(set(Tag_list))

In [206]:
from bokeh.layouts import row, column
from bokeh.models import CustomJS, Slider
from bokeh.plotting import figure, output_file, show, ColumnDataSource

In [207]:
revlog_cards.Tags.unique()

TypeError: unhashable type: 'list'

## Not Used 

col = Collection(COLLECTION_PATH)

dID_7 = col.decks.id('Exam 7')
dID_9 = col.decks.id('Exam 9')

##Exam 7
query= "select id,nid,did from cards where did = " + str(dID_7)
Exam_7_cards = col.db.all(query)

##Exam 9
query= "select id,nid,did from cards where did = " + str(dID_9)
Exam_9_cards = col.db.all(query)

col.close()